<a href="https://colab.research.google.com/github/Djinho/EvoNet-CNN-Insight/blob/main/FLOPS_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

Baseline

In [ ]:


# Define the model with the updated input shape
model = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005), padding='valid', input_shape=(198, 192, 1)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005), padding='valid'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005), padding='valid'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(units=128, activation='relu'),
    layers.Dense(units=1, activation='sigmoid')
])

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs and MACs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")
#2.721 × 10⁸.


FLOPs: 272125313


Recent and weak flops count

In [ ]:


def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 4
    filters = [39, 53, 94, 101]
    kernel_size = 2
    l1 = 7.734333542494201e-05
    l2 = 6.536870301639723e-05
    dense_units = 238
    dropout_rate = 0.29358511911026797
    learning_rate = 5.723121393552307e-05

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed
input_shape = (198, 192, 1)  # Example input shape

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs and MACs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")
#3.17954059×10 to he power 8


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


FLOPs: 317954059


recent and moderate flops count

In [ ]:


def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 5
    filters = [85, 119, 99, 115, 54]
    kernel_size = 3
    l1 = 0.00020959985382232806
    l2 = 0.00010741242396156945
    dense_units = 69
    dropout_rate = 0.29601406545155096
    learning_rate = 0.00013314057394016722

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed
input_shape = (198, 192, 1)  # Example input shape

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs and MACs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")
# 2.431 × 10⁹.

FLOPs: 2431044880


Intermeidate and weak flops

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 6
    filters = [36, 61, 125, 58, 46, 80]
    kernel_size = 2
    l1 = 5.7944957902052485e-05
    l2 = 0.00013078230073021723
    dense_units = 152
    dropout_rate = 0.24228689196387476
    learning_rate = 0.0003981287110754072

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed and input_shape is known
input_shape = (198, 192, 1)  # Example input shape

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs and MACs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")
# 3.637 × 10⁸.


FLOPs: 363706201


Intermeidate and moderate flops

In [ ]:


def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 3
    filters = [79, 83, 81]
    kernel_size = 2
    l1 = 2.4768594237307907e-05
    l2 = 0.0006992137199417164
    dense_units = 108
    dropout_rate = 0.18570184832201814
    learning_rate = 0.00018161195979508115

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed and input_shape is known
input_shape = (198, 192, 1)  # Replace with your actual input shape

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs and MACs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")
#6.671 × 10⁸.


FLOPs: 667099285


Intermediate strong

In [ ]:


def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 4
    filters = [124, 107, 37, 107]
    kernel_size = 3
    l1 = 0.00023342282213110015
    l2 = 6.698426243226582e-06
    dense_units = 205
    dropout_rate = 0.3223287518244954
    learning_rate = 0.00011551960587189376

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed and input_shape is known
input_shape = (198, 192, 1)  # Example input shape, update this if necessary

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")
#2.581 × 10⁹.


FLOPs: 2581355128


Ancient and weak

In [ ]:


def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 3
    filters = [79, 79, 125]
    kernel_size = 2
    l1 = 1.4238915610455433e-06
    l2 = 0.0011892049877332892
    dense_units = 227
    dropout_rate = 0.2605345852243278
    learning_rate = 2.5834311151381096e-05

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed
input_shape = (198, 192, 1)  # Replace with your actual input shape if different

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")


FLOPs: 725121082


Ancient and moderate flops

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 4
    filters = [50, 91, 73, 118]
    kernel_size = 2
    l1 = 0.0059151443315403895
    l2 = 1.1259329056013594e-05
    dense_units = 142
    dropout_rate = 0.33221970808788714
    learning_rate = 1.6120646535662953e-05

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed
input_shape = (198, 192, 1)  # Replace with your actual input shape if different

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")


FLOPs: 536664571


Ancient and strong flops count

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

def build_best_model(input_shape):
    # Best hyperparameters from Optuna optimization
    num_layers = 4
    filters = [88, 38, 88, 106]
    kernel_size = 3
    l1 = 1.5022150089120748e-05
    l2 = 0.0035801025371631446
    dense_units = 193
    dropout_rate = 0.2866844238417743
    learning_rate = 0.00016920857191232494

    # Building the model based on the best hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    return model

# Assuming gene_sim.data is already preprocessed
input_shape = (198, 192, 1)  # Replace with your actual input shape if different

# Build the model
model = build_best_model(input_shape)

# Convert model to concrete function
model_func = tf.function(lambda x: model(x))
model_func = model_func.get_concrete_function(tf.TensorSpec([1] + list(model.input_shape[1:])))

# Get frozen concrete function
frozen_func = convert_variables_to_constants_v2(model_func)
frozen_func.graph.as_graph_def()

# Calculate FLOPs
run_meta = tf.compat.v1.RunMetadata()
opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

# Use the profiler to calculate FLOPs
flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                      run_meta=run_meta, cmd='op', options=opts)

print(f"FLOPs: {flops.total_float_ops}")


FLOPs: 884400196
